# ___Les Macros___

---

- Cette partie traite très brièvement des macros en Rust et donne les principes généraux afin de mener à bien leur création. (Il ne s'agit pas ici de macro-procédurale).

## La macro Hello World (Exemple de RustByExample)

In [8]:
// Une simple macro nommée `say_hello_world`.
#[macro_export]
macro_rules! say_hello_world {
    // `()` indique que la macro ne prend aucun argument.
    () => (
        // La macro étendra le contenu de ce bloc.
        println!("Hello World!");
    )
}

fn main() {
    // Cet appel va étendre `println!("Hello World");`.
    say_hello_world!()
}
main()

Hello World!


()

**Sortie standard du programme :**
>```
>    Finished dev [unoptimized + debuginfo] target(s) in 0.19s
>    Running `target/debug/simple_macro`
> Hello World!
>```

**NB** Il existe des règles spécifiques d'export et d'import de macro en Rust depuis d'autres fichiers et des dépendances.

## Macro sur une seule ligne

In [33]:
/// Prints to the host through the serial interface, appending a newline.
#[macro_export]
macro_rules! serial_println {
    () => (print!("\n"));
    ($fmt:expr, $($arg:tt)*) => (print!(concat!($fmt, "\n"), $($arg)*));
    ($fmt:expr) => (print!(concat!($fmt, "\n")));
}

fn main() {
    /// Vide
    /// -> Juste un simple retour a la ligne
    serial_println!();

    /// Une chaine de caractere avec des arguments
    /// Affixher la chaine en developpant les arguments
    serial_println!("{} banane(s)", 42);

    // Une chaine de caractere seule
    // Afficher la chaine de caractere
    serial_println!("Toto21");
}
main()


42 banane(s)
Toto21


()

En entrée: Avant le `=>`

>- `()` veut dire vide.
>- `$var_name::expr` veut dire "expression formatée", chaine de caractère quoi.
>- la virgule `,` dans le second cas est le **délimiteur** utilise entre le 1er et le 2nd argument.
>- `$var_name:tt` veut dire arguments quelconque.
>- le wildcard `*` veut dire qu'il peut y en avoir toute une liste.

## Macro Complexe et explications

### Illustration

![COMPLEX](pictures/complex_macro.png)

### Utilisation de cette macro

```
safe_convertible_enum!(
    /// This list contains the sockets associated function types
    #[derive(Debug, Copy, Clone)]
    #[repr(u32)]
    enum CallType {
        /// Create an endpoint for communication
        SysSocket = 1,
        /// Bind a name to a socket
        SysBind = 2,
        /// Initiate a connection on a socket. Client connection-oriented
        SysConnect = 3,
        /// Listen for connections on a socket. Server connection-oriented
        SysListen = 4,
        /// Accept a connection on a socket. Server connection-oriented
        SysAccept = 5,
        /// Send a message on a socket. Similar to write with flags. connection-oriented
        SysSend = 9,
        /// Receive a message from a socket. Similar to read with flags. connection-oriented
        SysRecv = 10,
        /// Send a message on a socket. The destination address is specified. connectionless
        SysSendTo = 11,
        /// Receive a message from a socket. The source address is specified. connectionless
        SysRecvFrom = 12,
        /// Shut down part of a full-duplex connection. connection-oriented
        SysShutdown = 13,
    }
);
```

###  Le code de la macro, au cas où ...

In [34]:
/// Implements a new C style enum with his try_from boilerplate
macro_rules! safe_convertible_enum {
    (#[$main_doc:meta]
     #[$derive:meta]
     #[repr($primitive_type:tt)]
     enum $name:ident {
         $(
             #[$doc:meta]
             $variant:ident = $value:expr,
         )*
     }) => {
        #[$main_doc]
        #[$derive]
        #[repr($primitive_type)]
        pub enum $name {
            $(
                #[$doc]
                $variant = $value,
            )*
        }

        impl core::convert::TryFrom<$primitive_type> for $name {
            type Error = Errno;
            fn try_from(n: $primitive_type) -> Result<Self, Self::Error> {
                use $name::*;
                match n {
                    $($value => Ok($variant),)*
                    _ => Err(Errno::EINVAL),
                }
            }
        }
    }
}